In [1]:
import pandas as pd
import numpy as np

In [7]:
train_df = pd.read_csv('./data/train.csv')
train_df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'}, inplace=True)
train_df.head()

,ID,timestamp,item,corporation,location,supply,price
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [17]:
in_days = 30
pred_days = 28



seq_data = []

for i in train_df.item.unique():
    now_item = train_df.loc[train_df.item == i]
    for co in train_df.corporation.unique():
        now_co = now_item.loc[now_item.corporation == co]
        for lo in train_df.location.unique():
            now_lo = now_co.loc[now_co.location == lo]

            for i in range(in_days, len(now_lo) - pred_days):
                seq_data.append(list(now_lo.iloc[i]) + list(now_lo.price[i - in_days : i]) + list(now_lo.price[i + 1 : i + 1 + pred_days]))

seq_df = pd.DataFrame(seq_data, columns=[list(train_df.columns) + [f'price_t-{i}' for i in range(in_days, 0, -1)] + [f'price_t + {i}' for i in range(pred_days)]])

In [18]:
seq_df.head()

,ID,timestamp,item,corporation,location,supply,price,price_t-30,price_t-29,price_t-28,...,price_t + 18,price_t + 19,price_t + 20,price_t + 21,price_t + 22,price_t + 23,price_t + 24,price_t + 25,price_t + 26,price_t + 27
0,TG_A_J_20190131,2019-01-31,TG,A,J,15997.0,2278.0,0.0,0.0,1728.0,...,1581.0,2516.0,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0
1,TG_A_J_20190201,2019-02-01,TG,A,J,36010.0,2349.0,0.0,1728.0,1408.0,...,2516.0,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0
2,TG_A_J_20190202,2019-02-02,TG,A,J,8808.0,2286.0,1728.0,1408.0,1250.0,...,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0,4538.0
3,TG_A_J_20190203,2019-02-03,TG,A,J,0.0,0.0,1408.0,1250.0,0.0,...,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0,4538.0,0.0
4,TG_A_J_20190204,2019-02-04,TG,A,J,8468.0,1831.0,1250.0,0.0,1474.0,...,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0,4538.0,0.0,3992.0


In [19]:
seq_df.to_csv('30-28_train.csv', index=False)